##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reescrevendo automaticamente os símbolos das APIs TF 1.x e compat.v1

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/upgrade"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/upgrade.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/upgrade.ipynb"> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fonte no GitHub</a>
</td>
  <td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/upgrade.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>


Migrar para a API do TensorFlow 2.x das APIs do TF 1.x e `tf.compat.v1` envolve realizar várias alterações como reordenar argumentos, renomear símbolos e mudar os valores padrão para parâmetros. Realizar todas essas modificações manualmente não apenas é chato, como também é sujeito a erros. Para simplificar as alterações e deixar sua transição para o TF 2.x a mais perfeita possível, a equipe do TensorFlow criou o utilitário `tf_upgrade_v2` para ajudar na transição do código legado para a nova API.

Observação: `tf_upgrade_v2` é instalado automaticamente para o TensorFlow 1.13 e posterior (incluindo todos os builds do TF 2.x).

O uso típico está mostrado a seguir:

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">
tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt
</pre>

Isto vai acelerar seu processo de upgrade convertendo os scripts Python existentes do TensorFlow 1.x para o TensorFlow 2.x.

O script de conversão automatiza diversas transformações mecânicas da API, embora muitas APIs não possam ser migradas automaticamente. Ele também não é capaz de deixar seu código totalmente compatível com os comportamentos e APIs do TF2. Portanto, é apenas uma parte de sua jornada de migração.

## Módulos de compatibilidade

Determinados símbolos da API não podem ser atualizados simplesmente com uma substituição do string. Aqueles que não podem ser atualizados automaticamente serão mapeados para seus locais no módulo `compat.v1`. Este módulo substitui símbolos TF 1.x como `tf.foo` pela referência `tf.compat.v1.foo` equivalente. Se você já estiver usando APIs `compat.v1` através da importação do TF via `import tensorflow.compat.v1 as tf`, o script `tf_upgrade_v2` tentará converter esses usos para as APIs não compatíveis sempre que possível. Observe que, embora algumas APIs `compat.v1` sejam compatíveis com os comportamentos do TF2.x, muitas outras não são. Portanto, é recomendável revisar manualmente as substituições e migrá-las para novas APIs no namespace `tf.*` em vez do namespace `tf.compat.v1` o mais rápido possível.

Devido às descontinuações (deprecations) do módulo TensorFlow 2.x (por exemplo, `tf.flags` e `tf.contrib`), algumas alterações não podem ser contornadas mudando para `compat.v1`. A atualização desse código pode exigir o uso de uma biblioteca adicional (por exemplo, [`absl.flags`](https://github.com/abseil/abseil-py)) ou a mudança para um pacote em [tensorflow/addons](http://www.github.com/tensorflow/addons).


## Processo de atualização recomendado

O restante deste guia demonstra como usar o script de reescrita de símbolos. Embora o script seja fácil de usar, é altamente recomendável usá-lo como parte do seguinte processo:

1. **Teste de unidade**: certifique-se de que o código que você está atualizando tenha um conjunto de testes de unidade com cobertura razoável. Como trata-se de código Python, a linguagem não vai oferecer proteção contra diversas classes de erros. Certifique-se também de que qualquer dependência já tenha sido atualizada para ser compatível com o TensorFlow 2.x.

2. **Instale o TensorFlow 1.15**: faça o upgrade do seu TensorFlow para a versão mais recente do TensorFlow 1.x, pelo menos a versão 1.15. Isto inclui a API final do TensorFlow 2.0 em `tf.compat.v2`.

3. **Faça testes com a versão 1.15**: Certifique-se de que seus testes de unidade passem nesta etapa. Você vai executá-los repetidamente durante o upgrade, portanto, começar com os testes rodando no verde é importante.

4. **Execute o script de atualização**: execute `tf_upgrade_v2` em toda a sua árvore de código-fonte, inclusive testes. Isto vai atualizar seu código para um formato em que ele usará apenas os símbolos disponíveis no TensorFlow 2.0. Os símbolos descontinuados serão acessados ​​com `tf.compat.v1`. Eventualmente, eles exigirão atenção manual, mas não imediatamente.

5. **Execute os testes convertidos com o TensorFlow 1.15**: seu código ainda deve funcionar bem no TensorFlow 1.15. Execute seus testes de unidade novamente. Qualquer erro em seus testes nesta etapa significa que há um bug no script de upgrade. [Por favor, conte para a gente](https://github.com/tensorflow/tensorflow/issues) .

6. **Verifique se há avisos e erros no relatório de upgrade**: o script grava um arquivo de relatório que explica todas as conversões que você deve verificar novamente ou qualquer ação manual que precise ser executada. Por exemplo: Quaisquer instâncias restantes de contrib exigirão ação manual para serem removidas. Consulte [o RFC para obter mais instruções](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md).

7. **Instale o TensorFlow 2.x**: neste ponto, já deve ser seguro mudar para os binários do TensorFlow 2.x, mesmo que você esteja executando com comportamentos legados

8. **Teste com `v1.disable_v2_behavior`**: Executar novamente seus testes com `v1.disable_v2_behavior()` na função principal dos testes deve fornecer os mesmos resultados da execução na versão 1.15.

9. **Ative o comportamento V2**: agora que seus testes funcionam usando os binários do TF2, você pode começar a migrar seu código para evitar objetos `tf.estimator` e usar apenas os comportamentos TF2 suportados (sem desativar o comportamento TF2). Consulte os [Guias de migração](https://tensorflow.org/guide/migrate) para mais detalhes.

## Usando o script de reescrita de símbolos `tf_upgrade_v2`


### Configuração

Antes de começar, verifique se o TensorFlow 2.x está instalado.

In [ ]:
import tensorflow as tf

print(tf.__version__)

Clone o repositório git [tensorflow/models](https://github.com/tensorflow/models) para que você tenha código para testar:

In [ ]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

### Leia a ajuda

O script deve ser instalado com o TensorFlow. Aqui está a ajuda integrada:

In [ ]:
!tf_upgrade_v2 -h

### Exemplo de código TF1

Eis aqui um script simples em TensorFlow 1.0:

In [ ]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

Com o TensorFlow 2.x instalado ele não roda:

In [ ]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

### Único arquivo

O script pode ser executado em um único arquivo Python:

In [ ]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

O script imprimirá erros se não conseguir encontrar uma correção para o código. 

### Árvore de diretórios

Projetos típicos, incluindo este exemplo simples, irão conter muito mais de um único arquivo. Normalmente você vai querer fazer upgrade de um pacote inteiro, portanto, o script também pode ser executado numa árvore de diretórios:

In [ ]:
# update the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

Observe o aviso sobre a função `dataset.make_one_shot_iterator`.

Agora o script funciona com o TensorFlow 2.x:

Observe que, como o módulo `tf.compat.v1` está incluído no TF 1.15, o script convertido também roda no TensorFlow 1.15.

In [ ]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## Relatório detalhado

O script também relata uma lista de alterações em detalhes. Neste exemplo, ele encontrou uma transformação possivelmente insegura e incluiu um aviso na parte superior do arquivo: 

In [ ]:
!head -n 20 tree_report.txt

Observe novamente o aviso sobre a `Dataset.make_one_shot_iterator function`.

Em outros casos, a saída explicará o raciocínio para alterações não triviais:

In [ ]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [ ]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [ ]:
!cat dropout_report.txt

Aqui está o conteúdo do arquivo modificado, observe como o script adiciona nomes de argumento para lidar com argumentos movidos e renomeados:

In [ ]:
!cat dropout_v2.py

Um projeto maior poderá conter alguns erros. Por exemplo, converta o modelo deeplab:

In [ ]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

Produziu os arquivos de saída a seguir:

In [ ]:
!ls deeplab_v2

Mas houve erros. O relatório ajudará você a identificar o que precisa corrigir antes que isso seja executado. Aqui estão os três primeiros erros:

In [ ]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## Modo "Safety" (segurança)

O script de conversão também possui um modo `SAFETY` menos invasivo que simplesmente altera as importações para usar o módulo `tensorflow.compat.v1`:

In [ ]:
!cat dropout.py

In [ ]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [ ]:
!cat dropout_v2_safe.py

Como você pode ver, ele não atualiza seu código, mas permite que o código do TensorFlow 1 seja executado em binários do TensorFlow 2. Veja que isto não significa que seu código está executando comportamentos TF 2.x suportados!

## Ressalvas

- Não atualize partes de seu código manualmente antes de executar este script. Em particular, funções que tiveram argumentos reordenados como `tf.math.argmax` ou `tf.batch_to_space` fazem com que o script adicione incorretamente argumentos de palavras-chave que mapeiam incorretamente seu código existente.

- O script assume que `tensorflow` é importado usando `import tensorflow as tf` ou `import tensorflow.compat.v1 as tf`.

- Este script não reordena os argumentos. Em vez disso, o script adiciona argumentos de palavra-chave a funções que têm seus argumentos reordenados.

- Dê uma olhada em [tf2up.ml](https://github.com/lc0/tf2up) para obter uma ferramenta conveniente para atualizar notebooks Jupyter e arquivos Python em um repositório GitHub.

Para relatar bugs de script de atualização ou fazer solicitações de recursos, registre um issue no [GitHub](https://github.com/tensorflow/tensorflow/issues).